In [1]:
import requests
import os
import time
import pickle
from dotenv import load_dotenv
from collections import deque

In [2]:
load_dotenv()

DEFAULT_REGION_CODE = 'NA1'
DEFAULT_REGION = 'americas'
MY_PUUID = 'cgchml5db7FVLW6cfygY7dmICGAp6hU6OPeRX809alh99kdQW0m1I_Uy_RbVbNj0cB6EJZGL0K_5ew'
TEST_MATCH_ID = 'NA1_5031446976'
RATE_LIMITS_PER_SECOND = 20
RIOT_API_KEY = os.getenv("RIOT_API_KEY")

RGAPI-d791010a-f767-4046-93ad-2d9a2605e48b


In [60]:
#gets matches from users
def get_user_matches(puuid = None, region=DEFAULT_REGION):
    if not puuid:
        raise Exception('Please Select puuid')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')

In [61]:
#gets data from match
def get_match_data(match_id = None, region=DEFAULT_REGION):
    if not match_id:
        raise Exception('Please Select Match ID')
    
    api_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}'
    headers = {
        'X-Riot-Token' : RIOT_API_KEY,
    }

    try:
        response = requests.get(api_url,headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error Occured: {e}')


In [62]:
#starts with initial puuid, gets recent 20 matches from that puuid, and gets all unique users from those matches, repeat with the new unique users
def get_unique_users(initial_puuid = None, region=DEFAULT_REGION):
    unique_users_set = {initial_puuid}
    unique_users_list = list(unique_users_set)
    current_index = 0
    
    while len(unique_users_set)<=500:
        #cid stands for current user id 
        print(len(unique_users_set))
        cid = unique_users_list[current_index]
        current_index+=1

        try:
            match_history = get_user_matches(cid)
            for match_id in match_history:
                match_data = get_match_data(match_id)
                players = match_data['metadata']['participants']
                for player in players:
                    if player not in unique_users_set:
                        unique_users_set.add(player)
                        unique_users_list.append(player)
        
        except requests.exceptions.RequestException as e:
            print(f'Failed to retrieve data for user {cid}: {e}')
    return list(unique_users_set)

In [63]:
#gets first 20 matches of each player in unique players list
def get_unique_matches(unique_users):
    unique_matches_set = set()

    for i,puuid in enumerate(unique_users):
        if i % 100 == 0:
            print(len(unique_matches_set))
        try:
            match_history = get_user_matches(puuid)

            for match_id in match_history:
                unique_matches_set.add(match_id)
                    
        except requests.exceptions.RequestException as e:
            print(f'Failed to retrieve data for user {puuid}: {e}')
        
        time.sleep(1)
    return list(unique_matches_set)

In [111]:
#get list of unique users and match ids and save it for later use
unique_users = get_unique_users(MY_PUUID)
print(len(unique_users))

unique_matches = get_unique_matches(unique_users)
print(len(unique_matches))

#saving unique_users and unique_matches
with open('unique_users_1.pkl','wb') as f:
    pickle.dump(unique_users,f)

with open('unique_matches_1.pkl','wb') as f:
    pickle.dump(unique_matches,f)

In [83]:
#loading unique_users and unique_matches
with open('unique_users.pkl', 'rb') as f:
    unique_users_list = pickle.load(f)

with open('unique_matches.pkl', 'rb') as f:
    unique_matches_list = pickle.load(f)

#test load
print(len(unique_users_list))
print(len(unique_matches_list))

624
10726


In [64]:
def parse_data(match_data):
    region = match_data['info']['platformId']
    version = match_data["info"]["gameVersion"]
    game_mode = match_data["info"]["gameMode"]
    blue_team = [match_data['info']['participants'][i]['championName'] for i in range(5)]
    red_team = [match_data['info']['participants'][i]['championName'] for i in range(5, 10)]
    winning_team =  0 if match_data['info']['participants'][0]['win'] else 1
    
    return region, game_mode, version, blue_team, red_team, winning_team


In [69]:
def parse_match_list(match_list):
    parsed_match_data = []
    failed_matches = []  
    for match_id in match_list:
        try:
            match_data = get_match_data(match_id)
            if match_data['info']['gameMode'] in ('ARAM', 'CLASSIC'):
                parsed_data = parse_data(match_data)
                parsed_match_data.append(parsed_data)
        except Exception as e:
            print(f"An error occurred for match {match_id}: {e}")
            failed_matches.append(match_id)
        time.sleep(0.75)
    return parsed_match_data, failed_matches

In [70]:
parsed_match_data = parse_match_list(unique_matches_list)
with open('parsed_match_data.pkl','wb') as f:
    pickle.dump(parsed_match_data,f)

Error Occured: 503 Server Error: Service Unavailable for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5029386875
An error occurred for match NA1_5029386875: 'NoneType' object is not subscriptable


In [44]:
with open('parsed_match_data.pkl', 'rb') as f:
    parsed_match_data = pickle.load(f)

print(len(parsed_match_data))

20
